In [10]:
import sys
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

sys.path.append('..')
from audio_toolbox.metrics import splitTabularPredicting

audio_root_folder = '../archive/Data'
labels_csv = os.path.join(audio_root_folder, 'data.csv')
unslicedData = pd.read_csv(labels_csv, header=0)
slicedCSV = os.path.join(audio_root_folder, 'features.csv')
slicedDataSet = pd.read_csv(slicedCSV, header=0)
label = unslicedData['label']
unslicedData = unslicedData.sample(frac=1).reset_index(drop=True)

In [11]:
#from sklearn.preprocessing import StandardScaler

#scalar = StandardScaler()

In [12]:
#generate the testing data folds
K = 5
testDataFolds = []
length = len(unslicedData)
slice_size = length // K
start_index = 0
for i in range(K):
    end_index = min(start_index + slice_size, length)
    fold = unslicedData.iloc[start_index: end_index]
    start_index = end_index
    testDataFolds.append(fold)

In [13]:
#generate the training data folds
trainData = []
for i in range(K):
    index = testDataFolds[i]['index']
    trainData.append(slicedDataSet[~slicedDataSet['index'].isin(index)])

In [14]:
#model training
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = RandomForestClassifier(max_depth = 15, max_features = 5)
#model = LogisticRegression(max_iter=1000, C=1e-3)
#model = SVC(kernel='rbf', decision_function_shape='ovo', C=2)

predictions = []
testLabels = []
for i in range(K):
    X_train = trainData[i].copy()
    label = X_train['label']
    X_train.drop(['label','index'], axis=1, inplace=True)
    #npArray = X_train.values
    #npArray = scalar.fit_transform(npArray)
    #X_train = pd.DataFrame(npArray, columns=X_train.columns)
    model.fit(X_train, label)
    trainAccuracy = accuracy_score(model.predict(X_train), label)
    print(f"training accuracy: {trainAccuracy}")
    testData = testDataFolds[i].copy()
    testIndex = testData['index']
    testLabel = testData['label']
    testLabels.append(testLabel)
    #dataArray = testData.values
    #dataArray = scalar.transform(dataArray)
    #testData = pd.DataFrame(dataArray, columns=testData.columns)
    prediction= splitTabularPredicting(model, testIndex, slicedDataSet)
    predictions.append(prediction)

training accuracy: 0.9998748748748749
training accuracy: 0.9986237958213436
training accuracy: 1.0
training accuracy: 0.9987487487487487
training accuracy: 0.998623967975982


In [15]:
#average accuracy
accuracies = []
for i in range(K):
    prediction = predictions[i]
    accuracy = accuracy_score(testLabels[i], prediction)
    accuracies.append(accuracy)

mean_accuracy = np.mean(accuracies)
print(f"Mean Accuracy: {mean_accuracy}")

Mean Accuracy: 0.728643216080402
